<a href="https://colab.research.google.com/github/LordNJS/fai-practical/blob/main/Best_first_search.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Best First Search

In [ ]:
import copy
import heapq

class sudoku:
  # define possible states

  frontier = []
  explored_set = []
  possible_nums = [1,2,3,4,5,6,7,8,9]

  #heuristic counts how many rules are broken
  # the sudoku board fixed numbers are doubled, eg. 11,22,33,44
  # this is to easily find which are fixed and which can be changed after filling the board
  def heuristic(self,sudoku):
    rule_broken = 0
    for i in sudoku:
      # this is to not change sudoku when trying to get the fixed value
      rowl = [];
      for a in i:
        rowl.append(a%10)
      row = set(rowl)
      for a in row:
        rule_broken += rowl.count(a) - 1

    #get columns
    sudoku_cols = []
    for i in range(9):
      col = []
      for a in range(9):
        col.append(sudoku[a][i]%10)
      sudoku_cols.append(col)

    for i in sudoku_cols:
      col = set(i)
      for a in col:
        rule_broken += i.count(a) - 1

    #get squares
    sudoku_squares = []
    squares = [[],[],[]]
    for i in range(9):
      for j in range(9):
        squares[j//3].append(sudoku[i][j]%10)
      if i % 3 == 2:
        sudoku_squares += squares
        squares = [[],[],[]]

    for i in sudoku_squares:
      square = set(i)
      for a in square:
        rule_broken += i.count(a) - 1

    return rule_broken

  # get numbers that are not fixed (eg. fixed = 1,2; remaining_num = 3,4,5,6,7,8,9)
  def remaining_num(self,row):
    num_fixed = []
    #get fixed
    for i in row:
      if i > 10:
        num_fixed.append(i%10)

    nums = [1,2,3,4,5,6,7,8,9]
    for i in nums:
      if i in num_fixed:
        nums.remove(i)
    return nums

  #fill the board
  def fill(self,sudoku):
    for i in sudoku:
      for j in range(9):
        if i[j] != 0:
          i[j] = i[j] * 11

    for i in sudoku:
      add_num = self.remaining_num(i)
      for j in add_num:
        for x in range(len(i)):
          #fill if empty
          if i[x] == 0:
            i[x] = j
            break

    return sudoku

  def searching(self, sudoku, x, y):
    if sudoku[y][x] > 10:
      return sudoku

    new_sudoku = copy.deepcopy(sudoku)
    low_rule_broken = (99,[[]]);

    for a in self.possible_nums:
      new_sudoku[y][x] = a
      if a in self.explored_set:
        continue
      rule_broken = self.heuristic(new_sudoku)
      if (rule_broken,a) in self.frontier:
        continue
      heapq.heappush(self.frontier, (rule_broken, copy.deepcopy(new_sudoku)))
    return low_rule_broken

  def __init__(self, sudoku):
    if self.heuristic(sudoku) != 0:
      print("A rule is already broken")
      return

    sudoku = self.fill(sudoku)

    x = 0
    y = 0

    current_rule_broken = self.heuristic(sudoku)
    while current_rule_broken != 0:
      self.searching(sudoku,x,y)
      if self.frontier[0][0] < current_rule_broken:
        a = heapq.heappop(self.frontier)
        sudoku = a[1]
        self.explored_set.append(copy.deepcopy(a[1]))
        current_rule_broken = a[0]